# Introduzione allo sviluppo di interpreti

## Componenti principali da realizzare in un interprete

![FasiInteprete](files/images/fasi_interprete.png)

## Ciclo di interpretazione

L'interprete esegue le operazioni elementari del programma *una dopo l'altra*

Le fasi di *scanning/parsing* possono essere svolte a livello di:

* *intero programma*
* *singole istruzioni/espressioni*

Lo stesso vale per i *controlli di tipo* (e altre analisi)

* *analisi statica* (intero programma)
* *controlli dinamici* (per ogni istruzione/espressione, all'esecuzione)
* *analisi statica + controlli dinamici*

Ad esempio: L'interprete *JavaScript* (Node.js) prima esegue analizza la sintassi di *tutto il programma* e poi interpreta

```
console.log(1);
console.log(2;  //manca la parentesi
```

*Risultato:* non stampa `1`, perchè prima di eseguire controlla la sintassi di tutto il programma:

```
console.log(2;
            ^
SyntaxError: missing ) after argument list
```

Altro esempio: Il *toplevel di Ocaml* esegue il parsing ed esegue espressione per espressione

In [1]:
let x = 10 ;;
lett y = 20 ;; (* let scritto male... *)

val x : int = 10


error: compile_error

Prima ha parsato ed eseguito `let x = 10`, poi ha parsato `lett y = 20` trovando l'errore

## Un interprete (risolutore) di espressioni artimetiche

Sintassi delle espressioni aritmetiche (solo su interi, *non serve type checking*)

```
Exp ::= n | Exp op Exp | (Exp)
op ::= + | - | * | / 
```

Definiamo un tipo algebrico per rappresentare alberi di sintassi astratta (AST) per le espressioni definite da questa grammatica:

In [2]:
type op = Add | Sub | Mul | Div ;;
type exp =
    | Val of int
    | Op of op*exp*exp;;

type op = Add | Sub | Mul | Div


type exp = Val of int | Op of op * exp * exp


Un paio di esempi di espressioni:

```
exp1 = (3*7)-5
exp2 = 3*(7-5)
```

In [3]:
let exp1 = Op (Sub, (Op (Mul, Val 3, Val 7)), Val 5) ;;
let exp2 = Op (Mul, Val 3, (Op (Sub, Val 7, Val 5))) ;;

val exp1 : exp = Op (Sub, Op (Mul, Val 3, Val 7), Val 5)


val exp2 : exp = Op (Mul, Val 3, Op (Sub, Val 7, Val 5))


Scriviamo innanzitutto una funzione di utilità che (ri)trasforma l'AST in formato testuale:

In [4]:
let rec to_string e =
    let symbol o =
        match o with | Add -> "+" | Sub -> "-" | Mul -> "*" | Div -> "/"
    in 
    match e with
    | Val n -> string_of_int n
    | Op (o,e1,e2) -> "("^ (to_string e1) ^ (symbol o) ^ (to_string e2) ^")" ;;

val to_string : exp -> string = <fun>


In [5]:
to_string exp1 ;;
to_string exp2 ;; 

- : string = "((3*7)-5)"


- : string = "(3*(7-5))"


## Lo Scanner (o Lexer, o tokenizzatore)

Rappresentiamo ogni simbolo che può apparire in una espressione con un *token*

In [6]:
type token =
    | Tkn_NUM of int     (* numero n *)
    | Tkn_OP of string   (* operatore + - * / *)
    | Tkn_LPAR           (* simbolo ( *)
    | Tkn_RPAR           (* simbolo ) *)
    | Tkn_END;;          (* fine dell'espressione *)

type token =
    Tkn_NUM of int
  | Tkn_OP of string
  | Tkn_LPAR
  | Tkn_RPAR
  | Tkn_END


Lo *scanner* trasforma:

* la *rappresentazione testuale* dell'espressione (stringa)
* in una *lista di token*

Nell'eseguire questa trasformazione lo scanner *controlla* che nell'espressione non siano stati utilizzati *simboli non previsti*

Lo scanner non effettua (ancora) un controllo grammaticale:

* `)(3++(88` è corretta per lo scanner
* `[(3+2)-1]` non è corretta per lo scanner (`[` e `]` sono simboli non previsti)

### Implementazione dello scanner (funzione `tokenize`)

Scandisce ricorsivamente la stringa un carattere per volta e produce il token corrispondente

* Se trova più simboli numerici (`0-9`) uno dopo l'altro li *aggrega* in un unico token numerico (es. `Tkn_NUM 231`)
* Solleva l'eccezione `ParseError` se incontra un simbolo non previsto

In [7]:
exception ParseError of string*string;;

exception ParseError of string * string


In [8]:
(* scanner *)
let tokenize s =
    (* funzione che scandisce ricorsivamente s, 
       dove pos è la posizione del carattere corrente *)
    let rec tokenize_rec s pos =
        if pos=String.length s then [Tkn_END] (* caso base: fine lista *)
        else
            let c = String.sub s pos 1 (* estrae il carattere corrente *)
            in
            (* si richiama ricorsivamente prima di gestire il carattere corrente *)
            let tokens = tokenize_rec s (pos+1)
            in
                (* trasforma il carattere corrente in un token*)
                match c with
                | " " -> tokens
                | "(" -> Tkn_LPAR::tokens
                | ")" -> Tkn_RPAR::tokens
                | "+" | "-" | "*" | "/" -> (Tkn_OP c)::tokens
                | "0" | "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9" ->
                    (* accorpa cifre consecutive *)
                    (match tokens with
                    | Tkn_NUM n::tokens' -> 
                           Tkn_NUM (int_of_string (c^(string_of_int n)))::tokens'
                    | _ -> Tkn_NUM (int_of_string c)::tokens 
                    )
                | _ -> raise (ParseError ("Tokenizer","unknown symbol: "^c))    
    in
        tokenize_rec s 0 ;;    

val tokenize : string -> token list = <fun>


Esempio d'uso dello scanner:

In [9]:
let t1 = tokenize "(34 + 41) - (2223 * 2)";;

val t1 : token list =
  [Tkn_LPAR; Tkn_NUM 34; Tkn_OP "+"; Tkn_NUM 41; Tkn_RPAR; Tkn_OP "-";
   Tkn_LPAR; Tkn_NUM 2223; Tkn_OP "*"; Tkn_NUM 2; Tkn_RPAR; Tkn_END]


In [10]:
let t2 = tokenize "(34 + 41) - ([2223 * 2)";;

error: runtime_error

Funzioni di utilità per stampare un token o una lista di token:

In [11]:
let string_of_token t = 
    match t with
    | Tkn_NUM n -> "Tkn_NUM "^(string_of_int n)
    | Tkn_OP s -> "Tkn_OP "^s
    | Tkn_LPAR -> "Tkn_LPAR"
    | Tkn_RPAR -> "Tkn_RPAR"
    | Tkn_END -> "Tkn_END" ;;
    
let print_token t = 
    print_endline (string_of_token t);;
    
let print_tokenlist tl = 
    List.iter (fun t -> (print_token t)) tl;;

val string_of_token : token -> string = <fun>


val print_token : token -> unit = <fun>


val print_tokenlist : token list -> unit = <fun>


In [12]:
print_tokenlist t1;;  (* (34 + 41) - (2223 * 2) *)

Tkn_LPAR
Tkn_NUM 34
Tkn_OP +
Tkn_NUM 41
Tkn_RPAR
Tkn_OP -
Tkn_LPAR
Tkn_NUM 2223
Tkn_OP *
Tkn_NUM 2
Tkn_RPAR
Tkn_END


- : unit = ()


### "Problemi" di questa implementazione dello scanner

La funzione `tokenize_rec` è ricorsiva, ma *non tail-recursive*...

* potenziali problemi con espressioni estremamente lunghe

**ESERCIZIO:** Pensare ad *implementazioni alternative*:

* tail-recursive (ci vuole un accumulatore...)
* iterative (usando costrutti imperativi di OCaml)
* usando la funzione `String.iter` per trasformare ogni carattere in un token e la funzione `List.fold_left` per accorpare i token numerici consecutivi (o qualcosa di simile). Richiede due passate...

## Il Parser

Il parser

* *controlla* che l'espressione sia *sintatticamente corretta*
    * appartenga al linguaggio definito dalle regole BNF
* *genera* l'abstract syntax tree (*AST*)

Come si realizza un parser:

* la grammatica deve essere resa *non ambigua*
* se il linguaggio è *molto semplice*, si implementa un *parser a discesa ricorsiva*
* se il linguaggio *non è molto semplice*, si usa un *parser generator* (software che genera il codice del parser a partire dalla grammatica)
    * Flex/<a href="https://www.gnu.org/software/bison/">Bison</a> per generare un parser in C
    * <a href="http://pegjs.org/">PEG.js</a>, <a href="http://www.antlr.org/">ATNLR4</a> o <a href="https://github.com/zaach/jison">Jison</a> per generare un parser in JavaScript
    * <a href="https://ocaml.org/manual/lexyacc.html">ocamllex/ocamlyacc</a> o <a href="http://gallium.inria.fr/~fpottier/menhir/">Menhir</a> per generare un parser in OCaml
    * <a href="http://www.antlr.org/">ATNLR4</a> o <a href="https://javacc.github.io/javacc/">JavaCC</a> per generare un parser in Java
    * ...

Ridefiniamo la grammatica delle espressioni in modo *non ambiguo*

* Da così:
```
Exp ::= n | Exp op Exp | (Exp)
op ::= + | - | * | / 
```

* A così:
```
Exp ::= Term + Exp | Term - Exp | Term
Term ::= Factor * Term | Factor / Term | Factor
Factor ::= n | (Exp)
```
ossia:
```
Exp ::= Term [ + Exp | - Exp ]
Term ::= Factor [ * Term | / Term ]
Factor ::= n | (Exp)
```
dove `[...]` indica che il contenuto è opzionale


### Implementazione del parser (funzione `parse`)

Implementiamo un parser a discesa ricorsiva:

* una *funzione* per ogni *categoria sintattica* (`exp`, `term` e `factor`)
* le funzioni sono *mutuamente ricorsive*
* le funzioni si richiamano l'una con l'altra e *consumano token* secondo quanto indicato dalla grammatica
* ogni chiamata di funzione restituisce un nodo dell'AST
    * l'*AST* viene in questo modo corrisponde all'*albero delle chiamate*

In [13]:
(* parser *)
let parse s =

   (* usiamo un riferimento per scandire la lista dei token (ottenuta da tokenize) *)
    let tokens = ref (tokenize s) in
    
    (* restituisce il primo token senza rimuoverlo *)
    let lookahead () = match !tokens with 
        | [] -> raise (ParseError ("Parser","lookahead error"))
        | t::_ -> t
    in
    
    (* elimina il primo token *)
    let consume () = match !tokens with 
        | [] -> raise (ParseError ("Parser","consume error"))
        | t::tkns -> tokens := tkns
    in
    
    (* funzioni mutuamente ricorsive che seguono dalla grammatica *)
    (* Exp ::= Term [ + Exp | - Exp ] *)
    let rec exp () =
        let t1 = term() in
        match lookahead () with
        | Tkn_OP "+" -> consume(); Op (Add,t1,exp())
        | Tkn_OP "-" -> consume(); Op (Sub,t1,exp())
        | _ -> t1
        
    (* Term ::= Factor [ + Term | - Term ] *)
    and term () =
        let f1 = factor() in
        match lookahead() with
        | Tkn_OP "*" -> consume(); Op (Mul,f1,term())
        | Tkn_OP "/" -> consume(); Op (Div,f1,term())
        | _ -> f1

    (* Factor ::= n | ( Exp ) *)
    and factor () =
        match lookahead() with
        | Tkn_NUM n -> consume(); Val n
        | Tkn_LPAR -> consume(); let e = exp() in
                      (match lookahead() with
                      | Tkn_RPAR -> consume(); e
                      | _ -> raise (ParseError ("Parser","RPAR error"))
                      )
        | _ -> raise (ParseError ("Parser","NUM/LPAR error"))
    
    (* Si comincia chiamando exp che fa tutto il lavoro e restituisce la radice dell'AST *)
    in
        let ast = exp() in
        (* controlliamo che al termine sia rimasto solo Tkn_END *)
        match lookahead() with
        | Tkn_END -> ast
        | x -> print_tokenlist !tokens; raise (ParseError ("Parser","parse error"));;

val parse : string -> exp = <fun>


Esempi di esecuzione del parser:

In [14]:
let ast = parse "32 + 24 * 12 * (3-1) +2" ;;

val ast : exp =
  Op (Add, Val 32,
   Op (Add, Op (Mul, Val 24, Op (Mul, Val 12, Op (Sub, Val 3, Val 1))),
    Val 2))


In [15]:
let err = parse "16 + (7 - 2" ;;

error: runtime_error

### Discussione sui parser

Implementare un parser non è sempre così semplice:

* rendere la grammatica non ambigua non è sempre ovvio
* con la discesa ricorsiva è facile imbattersi in situazioni di *ricorsione infinita*
    * Es: se avessimo definito `Exp ::= Exp + Term | ...` la funzione `exp` per prima cosa si sarebbe chiamata ricorsivamente... 
* a volte non è sufficiente leggere un singolo token per capire quale regola grammaticale applicare
    * *lookhaed dinamico*
* ...

proprio per tutto questo: 
* esistono i *parser generator* (il cui funzionamento non è argomento di questo corso) 
* per i linguggi che vedremo *NON* implementeremo un parser
    * *partiremo dall'AST*

## L'interprete

E ora implementiamo l'interprete delle espressioni. 

Si parte dalla *definizione della semantica*.

L'abbiamo già definita almeno un paio di volte, in modi diversi:

* facciamo un po' di ordine...


### Structural Operational Semantics (SOS)

La definizione di una relazione di transizione per descrivere il comportamento dei programmi (o espressioni) scritti in un certo linguaggio segue solitamente l'approccio della *Structural Operational Semantics (SOS)*, o Semantica Strutturale Operazionale.

* *Semantics*: è una descrizione del "significato" del linguaggio. Nei linguaggi di programmazione il significato è dato dal comportamento dei programmi scritti in quel linguaggio
* *Operational*: descrive il comportamento dei programmi tramite una relazione di transizione ($\rightarrow$) che cattura le operazioni che vengono svolte passo-passo durante l'esecuzione
* *Structural*: la relazione di transizione è definita usando regole di inferenza basate sulla struttura sintattica (una o più regole per ogni costurutto definito dalla grammatica del linguaggio)

### Approcci alla definizione: small-step e big-step

Esistono due approcci principali alla definizione di una semantica in stile SOS

*Small-step* semantics: 

* ogni passo della relazione di transizione si esegue una singola operazione
* una computazione è una sequenza di passi
* esempio: $((3+(5*2))-1) \rightarrow_{ss} ((3+10)-1) \rightarrow_{ss} (13-1) \rightarrow_{ss} 12$

*LA COMPUTAZIONE SI SVILUPPA LUNGO LA SEQUENZA DI PASSI*

*Big-step* semantics:

* la relazione di transizione *descrive in un solo passo l'intera computazione*
* le singole operazioni sono descritte nell'albero di derivazione di quella transizione
* esempio: 
$$\frac {
    \frac{\frac{\vdots}{3 \rightarrow_{bs} 3} \quad \frac{\vdots}{(5*2) \rightarrow_{bs} 10} \quad 3+10=13}{(3+(5*2)) \rightarrow_{bs} 13} \quad 1 \rightarrow_{bs} 1 \quad 13-1 = 12}
{
((3+(5*2))-1) \rightarrow_{bs} 12
}
$$

*LA COMPUTAZIONE SI SVILUPPA DISCENDENDO L'ALBERO DI DERIVAZIONE*

### Semantica delle espressioni

Semantica *small-step*:

$$
n \rightarrow_{ss} n 
\qquad 
\frac{E_1 \rightarrow_{ss} E_1'}{E_1 \, op \, E_2 \rightarrow_{ss} E_1' \, op \, E_2} 
$$

$$
\frac{E_2 \rightarrow_{ss} E_2'}{n \, op \, E_2 \rightarrow_{ss} n \, op \, E_2'} 
\qquad
\frac{n_1 \mbox{op } n_2 = n}{n_1 \, op \, n_2 \rightarrow_{ss} n} 
$$

Semantica *big-step*:

$$
n \rightarrow_{bs} n
\qquad
\frac{E_1 \rightarrow_{bs} n_1 \quad E_2 \rightarrow_{bs} n_2 \quad n_1 \mbox{op } n_2 = n}{E_1 \, op \, E_1 \rightarrow_{bs} n}
$$

### Implementazione dell'inteprete (funzione `eval`) 

Consideriamo inizialmente la semantica *big-step* (più semplice).

$$
\tiny
n \rightarrow_{bs} n \qquad
\frac{E_1 \rightarrow_{bs} n_1 \quad E_2 \rightarrow_{bs} n_2 \quad n_1 \mbox{op } n_2 = n}{E_1 \, op \, E_1 \rightarrow_{bs} n}
$$

In [16]:
(* interprete big-step *)
let rec eval e =
    match e with
    | Val n -> Val n
    | Op (op,e1,e2) -> 
        (* chiamate ricorsive che calcolano le derivazioni per e1 ed e2 *)
        match (eval e1, eval e2) with
        | (Val n1, Val n2) -> (match op with   (* calcola n1 op n2 *)
                               | Add -> Val (n1+n2)
                               | Sub -> Val (n1-n2)
                               | Mul -> Val (n1*n2)
                               | Div -> Val (n1/n2)
                               )
        (* caso (inutile) aggiunto solo per rendere esaustivo il pattern matching *)
        | _ -> failwith "Errore impossibile che si verifichi" ;;

val eval : exp -> exp = <fun>


**NOTA:** l'interprete non restituisce un intero (es. `n`), ma un *valore* nella rappresentazione AST (es. `Val n`)

* per coerenza con $\rightarrow_{bs}$ che è definita su $Exp \times Val$ con $Val \subset Exp$.

Esempio d'uso dell'interprete:

In [17]:
(* AST dell'espressione 3+(5*2) *)
let exp = Op (Add , Val 3, Op ( Mul , Val 5, Val 2)) ;;

eval exp;;

val exp : exp = Op (Add, Val 3, Op (Mul, Val 5, Val 2))


- : exp = Val 13


### Implementazione dell'inteprete (funzione `eval_ss`)

Consideriamo ora la semantica *small-step*

$$
\tiny
n \rightarrow_{ss} n 
\qquad 
\frac{E_1 \rightarrow_{ss} E_1'}{E_1 \, op \, E_2 \rightarrow_{ss} E_1' \, op \, E_2} 
\qquad 
\frac{E_2 \rightarrow_{ss} E_2'}{n \, op \, E_2 \rightarrow_{ss} n \, op \, E_2'} 
\qquad
\frac{n_1 \mbox{op } n_2 = n}{n_1 \, op \, n_2 \rightarrow_{ss} n} 
$$

In [18]:
(* interprete small-step *)
let rec eval_ss e =
        match e with
        | Val n -> Val n
        | Op (op,e1,e2) ->
            (match (e1,e2) with
            (* se gli operandi sono entrambi valori, calcola n1 op n2 *)
            | (Val n1,Val n2) -> Val (match op with
                                      | Add -> n1 + n2    | Sub -> n1 - n2
                                      | Mul -> n1 * n2    | Div -> n1 / n2 )
            (* se e1 può fare un passo (cioè se è un Op e non un valore) *)
            | (Op (_,_,_),_) -> Op (op,(eval_ss e1),e2)
            (* altrimenti, se e1 è un valore ma e2 può fare un passo *)
            | (Val _,Op (_,_,_))  -> Op (op,e1,(eval_ss e2))
            ) ;;

val eval_ss : exp -> exp = <fun>


Esempio di esecuzione del nuovo interprete (small-step):

In [19]:
(* AST dell'espressione 3+(5*2) *)
let exp = Op (Add , Val 3, Op ( Mul , Val 5, Val 2)) ;;

eval_ss exp;;            (* fa un passo *)
eval_ss (eval_ss exp);;  (* fa due passi *)

val exp : exp = Op (Add, Val 3, Op (Mul, Val 5, Val 2))


- : exp = Op (Add, Val 3, Val 10)


- : exp = Val 13


Il risultato è lo stesso calcolato, in un solo passo, dall'interprete precedente (big-step):

In [43]:
eval exp ;;

- : exp = Val 13


## Dalla semantica SOS al codice, sistematicamente

Questi due esempi mostrano un *approccio sistematico di implementazione*:

1. Si usa il *pattern matching* per considerare i vari tipi di *nodo dell'AST*
2. Ogni tipo di *nodo* corrisponde a un *costrutto sintattico* definito dalla grammatica
3. Per ogni caso del pattern matching (costrutto sintattico) si *identificano le regole della semantica* relative ad esso (la semantica è syntax-driven)
4. Si verificano le *precondizioni* delle varie regole, possibilmente *richiamando ricorsivamente l'interprete* (per le regole che non sono assiomi)
5. Quando si trova una regola le cui precondizioni sono verificate, si calcola *risultato della transizione*

## Mettiamo le cose insieme

Scanner + Parser + Interprete

In [35]:
(* con inteprete big-step *)
let exec1 s = eval (parse s) ;;  

(* con inteprete small-step *)
let exec2 s = 
    let rec eval_rec ast = (* chiusura transitiva *)
        match ast with 
        | Val n -> Val n 
        | _ -> eval_rec ( eval_ss ast )
    in
        eval_rec (parse s) ;;

val exec1 : string -> exp = <fun>


val exec2 : string -> exp = <fun>


In [33]:
exec1 "3+2*(6-2)" ;;
exec2 "3+2*(6-2)" ;;

- : exp = Val 11


- : exp = Val 11


Piccola variante: interprete small-step che mostra tutti i passi:

In [41]:
let solve s =
    let rec solve_rec ast =
        match ast with
        | Val n -> (to_string ast)
        | _ -> (to_string ast)^" = "^(solve_rec (eval_ss ast)) 
    in
        solve_rec (parse s);;

solve "3+2*(6-2)" ;;

val solve : string -> string = <fun>


- : string = "(3+(2*(6-2))) = (3+(2*4)) = (3+8) = 11"
